In [2]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [3]:
from data_loaders.humanml.motion_loaders.model_motion_loaders import get_mdm_loader  # get_motion_loader
from utils.fixseed import fixseed
from utils import dist_util
from diffusion import logger
from utils.parser_util import eval_args
from eval.eval_humanml import load_dataset, create_model_and_diffusion, load_saved_model, evaluation
from model.cfg_sampler import ClassifierFreeSampleModel
from data_loaders.humanml.networks.evaluator_wrapper import EvaluatorMDMWrapper
import torch
from data_loaders.humanml.data.dataset import abs3d_to_rel, sample_to_motion

/home/epinyoan/miniconda3/envs/gmd/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sys.argv = ['/home/epinyoan/git/guided-motion-diffusion/eval/eval_humanml_condition.py', 
            '--model_path', 
            './save/unet_adazero_xl_x0_abs_proj10_fp16_clipwd_224/model000500000.pt']
args = eval_args()
args.batch_size = 32 # This must be 32! Don't change it! otherwise it will cause a bug in R precision calc!
args.num_frames = 196 # This must be 196!
args.gen_two_stages = True
skip_first_stage = False
if_ddim = "_ddim" if args.use_ddim else ""
fixseed(args.seed)

name = os.path.basename(os.path.dirname(args.model_path))
niter = os.path.basename(args.model_path).replace('model', '').replace('.pt', '')
# log_file = os.path.join(os.path.dirname(args.model_path), 'eval_humanml_cond_{}_{}'.format(name, niter))
log_file = os.path.join(os.path.dirname(args.model_path), 'eval_humanml_cond_{}_{}_{}'.format(name, niter, if_ddim))

# save_dir = os.path.join(os.path.dirname(args.model_path), f'eval_cond_{niter}{"_avg" if flag.GEN_USE_AVG_MODEL else ""}')
save_dir = os.path.join(os.path.dirname(args.model_path), f'eval_cond_{niter}{if_ddim}')
print('> Saving the generated motion to {}'.format(save_dir))

# NOTE: Set the final line imputing step t here. This is when we stop imputing with point-to-point trajectory
# and start imputing with key locations only
impute_until = 100  # int(args.impute_until) # 100
print('> Impute the trajectory with point-to-point until t = {}'.format(impute_until))
# skip_first_stage = True  # bool(args.skip_first) # True
print('> Skip first stage = {}'.format(skip_first_stage))

args.eval_mode = 'wo_mm'
print(f'Eval mode [{args.eval_mode}]')

> Saving the generated motion to ./save/unet_adazero_xl_x0_abs_proj10_fp16_clipwd_224/eval_cond_000500000
> Impute the trajectory with point-to-point until t = 100
> Skip first stage = False
Eval mode [wo_mm]


In [5]:
if args.eval_mode == 'wo_mm':
    num_samples_limit = 1000
    # num_samples_limit = 32
    run_mm = False
    mm_num_samples = 0
    mm_num_repeats = 0
    mm_num_times = 0
    diversity_times = 300
    # diversity_times = 10
    # replication_times = 2
    replication_times = 20  # about 12 Hrs
else:
    raise Exception('Not support', '/home/epinyoan/git/guided-motion-diffusion/eval/eval_humanml.py')

dist_util.setup_dist(args.device)
logger.configure()

logger.log("creating data loader...")
split = 'test'
gt_loader = load_dataset(args, args.num_frames, split, hml_mode='gt')
gen_loader = load_dataset(args, args.num_frames, split, hml_mode='eval')
num_actions = gen_loader.dataset.num_actions

Logging to /tmp/openai-2024-05-29-19-07-32-642271
creating data loader...


Reading ././dataset/humanml_opt.txt
Loading dataset t2m ...
proj_matrix_dir = ./dataset
mode = gt
t2m dataset aug: none std_scale_shift: (1.0, 0.0)
t2m dataset drop redundant information: False


100%|██████████| 4384/4384 [00:02<00:00, 2018.17it/s]


Pointer Pointing at 0
Loading random projection matrix from ./dataset
Reading ././dataset/humanml_opt.txt
Loading dataset t2m ...
proj_matrix_dir = ./dataset
mode = eval
t2m dataset aug: none std_scale_shift: (1.0, 0.0)
t2m dataset drop redundant information: False


100%|██████████| 4384/4384 [00:01<00:00, 2355.90it/s]


Pointer Pointing at 0
Loading random projection matrix from ./dataset


In [6]:
from eval.eval_humanml_condition import load_model
from data_loaders.humanml.motion_loaders.model_motion_loaders import get_mdm_loader, get_mdm_loader_cond  # get_motion_loader

In [17]:
motion_model, motion_diffusion, traj_model, traj_diffusion = load_model(args, gen_loader)
model_dict = {"motion": motion_model, "traj": traj_model}
diffusion_dict = {"motion": motion_diffusion, "traj": traj_diffusion}
eval_motion_loaders = {
    ################
    ## HumanML3D Dataset##
    ################
    'vald':
    lambda seed: get_mdm_loader_cond(
        model_dict, diffusion_dict, args.batch_size,
        gen_loader, mm_num_samples, mm_num_repeats, gt_loader.dataset.opt.max_motion_length, num_samples_limit, args.guidance_param,
        seed=seed,
        save_dir=save_dir,
        impute_until=impute_until,
        skip_first_stage=skip_first_stage,
        use_ddim=args.use_ddim,
    )
}

Creating model and diffusion...
nfeats: 1
Using UNET with lantent dim:  512  and mults:  [2, 2, 2, 2]
dims:  [263, 1024, 1024, 1024, 1024] mults:  [2, 2, 2, 2]
[ models/temporal ] Channel dimensions: [(263, 1024), (1024, 1024), (1024, 1024), (1024, 1024)]
EMBED TEXT
Loading CLIP...
Loading checkpoints from [./save/unet_adazero_xl_x0_abs_proj10_fp16_clipwd_224/model000500000.pt]...
loading avg model
Setting traj model ...
 - model_path is given in the function call. Cmd path, if any, will be ignored.
args_traj.traj_only____ True
nfeats: 1
Using UNET with lantent dim:  512  and mults:  [0.125, 0.25, 0.5]
dims:  [4, 64, 128, 256] mults:  [0.125, 0.25, 0.5]
[ models/temporal ] Channel dimensions: [(4, 64), (64, 128), (128, 256)]
EMBED TEXT
Loading CLIP...
Loading traj model checkpoints from [./save/traj_unet_adazero_swxs_eps_abs_fp16_clipwd_224/model000062500.pt]...
loading avg model


In [ ]:
eval_wrapper = EvaluatorMDMWrapper(args.dataset, dist_util.dev())
evaluation(eval_wrapper, gt_loader, eval_motion_loaders, log_file, replication_times, diversity_times, mm_num_times, run_mm=run_mm)

### CompMDMGeneratedDatasetCondition

In [8]:
from sample.condition import (cond_fn_key_location, get_target_from_kframes, get_target_and_inpt_from_kframes_batch, 
                              log_trajectory_from_xstart, get_inpainting_motion_from_traj, get_inpainting_motion_from_gt,
                              cond_fn_key_location, compute_kps_error, cond_fn_sdf,
                              CondKeyLocations, CondKeyLocationsWithSdf)
import copy
from data_loaders.humanml.scripts.motion_process import recover_from_ric
from functools import partial

In [9]:
model_device = next(motion_model.parameters()).device
motion_diffusion, traj_diffusion = diffusion_dict["motion"], diffusion_dict["traj"]
clip_denoised = True
abs_3d = True
trajectory_classifier_scale = 100.0 # 100.0
sample_fn_traj = (
    traj_diffusion.p_sample_loop if not args.use_ddim else traj_diffusion.ddim_sample_loop
)
sample_fn_motion = (
    motion_diffusion.p_sample_loop if not args.use_ddim else motion_diffusion.ddim_sample_loop
)

motion, model_kwargs = next(iter(gen_loader))
tokens = [t.split('_') for t in model_kwargs['y']['tokens']]
model_kwargs['y']['scale'] = torch.ones(motion.shape[0], device=dist_util.dev()) * args.guidance_param

### 1. Prepare motion for conditioning

In [10]:
### 1. Prepare motion for conditioning ###
traj_model_kwargs = copy.deepcopy(model_kwargs)
traj_model_kwargs['y']['traj_model'] = True
model_kwargs['y']['traj_model'] = False

# Convert to 3D motion space
# NOTE: the 'motion' will not be random projected if dataset mode is 'eval' or 'gt', 
# even if the 'self.dataset.t2m_dataset.use_rand_proj' is True
gt_poses = motion.permute(0, 2, 3, 1)
gt_poses = gt_poses * gen_loader.dataset.std + gen_loader.dataset.mean  # [bs, 1, 196, 263]
# (x,y,z) [bs, 1, 120, njoints=22, nfeat=3]
gt_skel_motions = recover_from_ric(gt_poses.float(), 22, abs_3d=False)
gt_skel_motions = gt_skel_motions.view(-1, *gt_skel_motions.shape[2:]).permute(0, 2, 3, 1)
gt_skel_motions = motion_model.rot2xyz(x=gt_skel_motions, mask=None, pose_rep='xyz', glob=True, translation=True, 
                                    jointstype='smpl', vertstrans=True, betas=None, beta=0, glob_rot=None, get_rotations_back=False)

# Next, sample points, then prepare target and inpainting mask for trajectory model
## Sample points
n_keyframe = 5
sampled_keyframes = torch.rand(motion.shape[0], n_keyframe) * model_kwargs['y']['lengths'].unsqueeze(-1)
sampled_keyframes = torch.floor(sampled_keyframes).int().sort()[0]

## Prepare target and mask for grad cal
# Prepare trajecotry inpainting
(target, target_mask, 
inpaint_traj, inpaint_traj_mask,
inpaint_traj_points, inpaint_traj_mask_points,
inpaint_motion, inpaint_mask, 
inpaint_motion_points, inpaint_mask_points) = \
    get_target_and_inpt_from_kframes_batch(gt_skel_motions, 
                                           sampled_keyframes, 
                                           gen_loader.dataset)

target = target.to(model_device)
target_mask = target_mask.to(model_device)
model_kwargs['y']['target'] = target
model_kwargs['y']['target_mask'] = target_mask

impute_slack = 20
# NOTE: For debugging
############################ ??????????????? ####################
# traj_model_kwargs['y']['log_name'] = self.save_dir
# traj_model_kwargs['y']['log_id'] = i
# model_kwargs['y']['log_name'] = self.save_dir
# model_kwargs['y']['log_id'] = i
# motion model always impute until 20
#######################################################
model_kwargs['y']['cond_until'] = impute_slack
model_kwargs['y']['impute_until'] = impute_slack

### Add motion to inpaint
traj_model_kwargs['y']['inpainted_motion'] = inpaint_traj.to(model_device) # init_motion.to(model_device)
traj_model_kwargs['y']['inpainting_mask'] = inpaint_traj_mask.to(model_device)

# Set when to stop imputing
traj_model_kwargs['y']['cond_until'] = impute_slack
traj_model_kwargs['y']['impute_until'] = impute_until
# NOTE: We have the option of switching the target motion from line to just key locations
# We call this a 'second stage', which will start after t reach 'impute_until'
traj_model_kwargs['y']['impute_until_second_stage'] = impute_slack
traj_model_kwargs['y']['inpainted_motion_second_stage'] = inpaint_traj_points.to(model_device)
traj_model_kwargs['y']['inpainting_mask_second_stage'] = inpaint_traj_mask_points.to(model_device)


In [11]:
trajectory_classifier_scale

100.0

In [12]:
target.shape

torch.Size([32, 196, 22, 3])

In [ ]:
sample_traj = sample_fn_traj(
    traj_model,
    inpaint_traj.shape,
    clip_denoised=clip_denoised,
    model_kwargs=traj_model_kwargs,  # <-- traj_kwards
    skip_timesteps=0,  # NOTE: for debugging, start from 900
    init_image=None,
    progress=True,
    dump_steps=None,
    noise=None,
    const_noise=False,
    cond_fn=partial(
        cond_fn_key_location, # cond_fn_sdf, #,
        transform=gen_loader.dataset.t2m_dataset.transform_th,
        inv_transform=gen_loader.dataset.t2m_dataset.inv_transform_th,
        target=target,
        target_mask=target_mask,
        kframes=[],
        abs_3d=abs_3d, # <<-- hard code,
        classifiler_scale=trajectory_classifier_scale,
        use_mse_loss=False),  # <<-- hard code
)   

100%|██████████| 1000/1000 [00:52<00:00, 18.93it/s]


In [ ]:
traj_motion, traj_mask = get_inpainting_motion_from_traj(
    sample_traj, inv_transform_fn=gen_loader.dataset.t2m_dataset.inv_transform_th)
# Get target for loss grad
# Target has dimention [bs, max_motion_length, 22, 3]
target = torch.zeros([motion.shape[0], gt_loader.dataset.opt.max_motion_length, 22, 3], device=traj_motion.device)
target_mask = torch.zeros_like(target, dtype=torch.bool)
# This assume that the traj_motion is in the 3D space without normalization
# traj_motion: [3, 263, 1, 196]
target[:, :, 0, [0, 2]] = traj_motion.permute(0, 3, 2, 1)[:, :, 0,[1, 2]]
target_mask[:, :, 0, [0, 2]] = True
# Set imputing trajectory
model_kwargs['y']['inpainted_motion'] = traj_motion
model_kwargs['y']['inpainting_mask'] = traj_mask

In [ ]:
cond_fn = None
sample_motion = sample_fn_motion(
    motion_model,
    (motion.shape[0], motion_model.njoints, motion_model.nfeats, motion.shape[3]),  # motion.shape
    clip_denoised=clip_denoised,
    model_kwargs=model_kwargs,
    skip_timesteps=0,
    init_image=None,
    progress=True,
    dump_steps=None,
    noise=None,
    const_noise=False,
    cond_fn=cond_fn
        # partial(
        # cond_fn_key_location,
        # transform=self.dataset.t2m_dataset.transform_th,
        # inv_transform=self.dataset.t2m_dataset.inv_transform_th,
        # target=target,
        # target_mask=target_mask,
        # kframes=[],
        # abs_3d=True, # <<-- hard code,
        # classifiler_scale=motion_classifier_scale,
        # use_mse_loss=False),  # <<-- hard code
)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [03:22<00:00,  4.95it/s]


In [ ]:
sample = sample_motion[:, :, :, :gt_loader.dataset.opt.max_motion_length]
cur_motion = sample_to_motion(sample, gen_loader.dataset, motion_model)

In [ ]:
from exit.utils import animate3d, t2m_bone
animate3d(cur_motion[3].permute(2,0,1).cpu().detach().numpy(), BONE_LINK=t2m_bone, first_total_standard=63, axis_visible=True)